# KEN3140: Lab 4 (Part 2)

### Writing and executing basic SPARQL queries on a local RDF file

##### Authors:
+ [Vincent Emonet](https://www.maastrichtuniversity.nl/vincent.emonet): [vincent.emonet@maastrichtuniversity.nl](mailto:vincent.emonet@maastrichtuniversity.nl)
+ [Kody Moodley](https://www.maastrichtuniversity.nl/kody.moodley): [kody.moodley@maastrichtuniversity.nl](mailto:kody.moodley@maastrichtuniversity.nl)

##### Affiliation: 
[Institute of Data Science](https://www.maastrichtuniversity.nl/research/institute-data-science)

##### License:
[CC-BY 4.0](https://creativecommons.org/licenses/by/4.0)

##### Date:
2021-09-06

In addition to running SPARQL queries on a remote SPARQL endpoint on the Web, you can also execute SPARQL queries on RDF documents that you store locally on a file.

We will use the Java RDF library [Apache Jena](https://jena.apache.org/) to demonstrate this. Follow these instructions to install the library:

1. Copy the `apache-jena-3.16.0` folder from Lab 3 to the same directory as this notebook
2. Run the next cell (it will import all jars in the `apache-jena-3.16.0/lib` folder)

In [3]:
%jars apache-jena-*/lib/*.jar

In [4]:
import org.apache.jena.riot.RDFDataMgr;
import org.apache.jena.rdf.model.*;
import org.apache.jena.util.PrintUtil;
import org.apache.jena.vocabulary.RDF;
import org.apache.jena.query.Query;
import org.apache.jena.query.QueryExecution;
import org.apache.jena.query.QueryExecutionFactory;
import org.apache.jena.query.QueryFactory;
import org.apache.jena.query.QuerySolution;
import org.apache.jena.query.ResultSetFormatter;
import org.apache.jena.query.ResultSet;
import java.io.File;
import org.apache.commons.io.FileUtils;
import java.nio.charset.StandardCharsets;

### Setup: load the RDF file in an Apache Jena model

Loads the RDF file, and creates a `runSparqlQuery()` function to run SPARQL queries in a single call from a string or a file.

In [5]:
Model model = RDFDataMgr.loadModel("family.ttl");

In [6]:
static void runSparqlQuery(String queryString, Model model) throws java.io.IOException {
    if (queryString.endsWith(".rq")) {
        queryString = FileUtils.readFileToString(new File(queryString), StandardCharsets.UTF_8);
    }
    System.out.println(queryString);
    Query query = QueryFactory.create(queryString);
    QueryExecution qexec = QueryExecutionFactory.create(query, model);
    ResultSetFormatter.out(qexec.execSelect(), model);
} 

### Write down a SPARQL query in a Java string

Java does not support multiline strings, which is why we need to use "+" to compile strings in a readable way here below.\
We create a basic SPARQL select query to list all entities and their types in the graph (for those entities that actually specify one).

In [12]:
String queryString = "PREFIX schema: <https://schema.org/> \n" +
    "SELECT * WHERE{ \n" + 
    "?entity a ?type .\n" +
    "}";

Now we execute this query on our model and display the results

In [13]:
runSparqlQuery(queryString, model)

PREFIX schema: <https://schema.org/> 
SELECT * WHERE{ 
?entity a ?type .
}
----------------------------------
| entity         | type          |
| family:Miranda | schema:Person |
| family:Pierre  | schema:Person |
----------------------------------


## Import a SPARQL query from a file

You can also import a SPARQL query to execute from a file. This might be useful if you have a list of predefined SPARQL queries and you don't want to rewrite them each time. We have provided an example SPARQL query in a file **with the extension `.rq`**

In [24]:
runSparqlQuery("query.rq", model)

PREFIX schema: <https://schema.org/>
PREFIX family: <https://my-family.org/>
SELECT * 
WHERE {
#     ?entity a ?type .                 # Line 1
    ?entity a schema:Person ;         # Line 2
              schema:parent ?parent . # Line 3
}
-------------------
| entity | parent |
-------------------


## Task 1: 
Comment out Line 1 ``query.rq`` and uncomment Lines 2-3. What inferences do you expect to receive for this query? After writing down what you think the results would be, execute the previous cell again to run the query again. Is this the result that you expected? The reason you get this unexpected answer is that Apache Jena "Model" class does not enable inference when running SPARQL queries.

### Task 2: 
By default, Apache Jena does not apply inference with SPARQL queries. In order to use inference, you need to explicitly indicate this by creating an instance of "InfModel" rather than the basic "Model".

In [25]:
InfModel rdfsModel = ModelFactory.createRDFSModel(RDFDataMgr.loadModel("family.ttl"));

Now, repeat Task 1 by commenting out the Line 1 in ``query.rq`` and uncomment Lines 2-3. Then, run the next cell and observe the results.

In [27]:
runSparqlQuery("query.rq", rdfsModel)

PREFIX schema: <https://schema.org/>
PREFIX family: <https://my-family.org/>
SELECT * 
WHERE {
#     ?entity a ?type .                 # Line 1
    ?entity a schema:Person ;         # Line 2
              schema:parent ?parent . # Line 3
}
------------------------------------
| entity         | parent          |
| family:Miranda | family:Mathilde |
| family:Miranda | family:Pierre   |
------------------------------------


Now we get expected results!